In [1]:
import numpy as np 
import matplotlib.pyplot as pl
import pandas as pd

In [2]:
data_russian = pd.read_csv("ussr_russia.csv",sep=";")
data_usa=pd.read_csv("usa.csv",sep=";")
data_bloggers=pd.read_csv("bloggers.csv",sep=";")

In [3]:
data_russian.describe()


,id
count,10442.000000
mean,15049.323980
std,12810.734448
min,0.000000
25%,4122.250000
50%,11759.500000
75%,23021.750000
max,47472.000000


In [4]:
data_usa.describe()

,id
count,6049.000000
mean,26972.087948
std,17028.247594
min,1.000000
25%,12416.000000
50%,25316.000000
75%,41060.000000
max,59491.000000


In [5]:
data_bloggers.describe()

,id
count,419.000000
mean,234.124105
std,135.415253
min,0.000000
25%,115.500000
50%,236.000000
75%,351.500000
max,469.000000


In [6]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
import keras
import cv2
import os
import shutil
from PIL import Image

In [7]:
def move_images_with_annotations(root_folder, target_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            image_path = os.path.join(root, file[:-4]+".jpg")
            annotation_path = os.path.join(root, file[:-4] + ".txt")  
            target_subfolder = os.path.join(target_folder, os.path.relpath(root, root_folder))
            os.makedirs(target_subfolder, exist_ok=True)

            with open(annotation_path, "r") as f:
                for line in f:
                    params = line.split()
                    if len(params) >= 5:  # Проверяем, что есть хотя бы 5 значений в строке
                        center_x, center_y, width, height = map(float, params[1:5])
                        x1 = float(center_x - width / 2)
                        y1 = float(center_y - height / 2)
                        x2 = float(center_x + width / 2)
                        y2 = float(center_y + height / 2)
                        
                        image = cv2.imread(image_path)
                        cropped_image = image[int(image.shape[0]*y1):int(image.shape[0]*y2), int(image.shape[1]*x1):int(image.shape[1]*x2)]
                        target_image_path = os.path.join(target_subfolder, file[:-4] + "_cropped.jpg")

                        if cropped_image.size == 0:  # Проверяем, что размерность массива не равна нулю
                            continue
                        
                        cv2.imwrite(target_image_path, cropped_image)
                        
                        break

            # with open(annotation_path, "r") as f:
            #     line=f.readline()
            #     params = line.strip().split()
            #     center_x, center_y, width, height = map(float, params[1:5])
            #     x1 = float(center_x - width / 2)
            #     y1 = float(center_y - height / 2)
            #     x2 = float(center_x + width / 2)
            #     y2 = float(center_y + height / 2)
                
            #     image = cv2.imread(image_path)
            #     cropped_image = image[int(image.shape[1]*y1):int(image.shape[1]*y2), int(image.shape[0]*x1):int(image.shape[0]*x2)]
            #     target_image_path = os.path.join(target_subfolder, file[:-4] + "_cropped.jpg")
            #     cv2.imwrite(target_image_path, cropped_image)
            # cv2.imwrite(image_path,cropped)
            # shutil.copy(cropped, target_subfolder)
            # shutil.copy(annotation_path, target_subfolder)

            

        


In [11]:
from imutils import paths
import face_recognition
import pickle
import cv2
import os

In [13]:
imagePaths = list(paths.list_images('bloggers_cropped_new'))
knownEncodings = []
knownNames = []
# перебираем все папки с изображениями
for (i, imagePath) in enumerate(imagePaths):
    # извлекаем имя человека из названия папки
    name = imagePath.split(os.path.sep)
    # загружаем изображение и конвертируем его из BGR (OpenCV ordering)
    # в dlib ordering (RGB)
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # используем библиотеку Face_recognition для обнаружения лиц
    boxes = face_recognition.face_locations(rgb, model='hog')
    # вычисляем эмбеддинги для каждого лица
    encodings = face_recognition.face_encodings(rgb, boxes)
    # loop over the encodings
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)
# сохраним эмбеддинги вместе с их именами в формате словаря
data = {"encodings": knownEncodings, "names": knownNames}
# для сохранения данных в файл используем метод pickle
f = open("face_enc", "w")
f.write(data)
f.close()

TypeError: write() argument must be str, not bytes

In [14]:
data_russian.head()

,id,name,link,specialization,films_years,description,path_to_folder
0,3250,Полина Лазарева,https://www.kinopoisk.ru/name/1691869,"Актриса,Режиссер,Сценарист","21, 1997 — 2024","Операция «Мухаббат»,Екатерина,Город,Скажи правду",actors_ussr_russia/3250
1,8125,Александр Мосолов,https://www.kinopoisk.ru/name/2201119,Актер,"85, 1993 — 2023","Летчик,Лихач,Ментовские войны 8,Первый отдел,Н...",actors_ussr_russia/8125
2,1625,Евгения Лютая,https://www.kinopoisk.ru/name/1671773,Актриса,"44, 2000 — 2022","Охотники за караванами,Дети Арбата,Склифосовск...",actors_ussr_russia/1625
3,23692,Никита Тезин,https://www.kinopoisk.ru/name/1746570,Актер,"45, 2004 — 2023","Битва за Севастополь,Молодая гвардия,Жизнь и с...",actors_ussr_russia/23692
4,44589,Янис Кирмушка,https://www.kinopoisk.ru/name/1124881,Актер,"4, 2005 — 2016",NaN,actors_ussr_russia/44589


In [17]:
from sklearn.decomposition import PCA
from scipy.spatial import distance

In [25]:
pca = PCA(n_components=16)
data_vector_blogger=pd.read_csv("bloggers_vector.csv")
features = data_vector_blogger.values
pca.fit(features)
pca_features = pca.transform(features)

In [26]:
pca_features

array([[-2.04749997e+03, -1.56861279e+00, -2.51767399e-02, ...,
         3.81821939e-02,  1.46279503e-02, -3.90285788e-02],
       [-2.04649997e+03, -1.55327123e+00, -4.28734113e-02, ...,
         7.92004921e-02,  2.91834430e-02, -2.42722684e-02],
       [-2.04549998e+03, -1.49846592e+00,  4.94034933e-03, ...,
        -4.16010234e-02,  5.04121682e-02, -2.72761386e-02],
       ...,
       [ 2.04550002e+03, -1.48115432e+00,  3.61891554e-02, ...,
         7.54126525e-03,  1.41177613e-02, -2.01340402e-02],
       [ 2.04650002e+03, -1.51706367e+00,  3.36086550e-02, ...,
         3.73419038e-02,  1.21743614e-02, -1.40933685e-02],
       [ 2.04750000e+03,  2.23361012e-01, -1.03442541e+00, ...,
        -3.00176072e-01,  7.15080107e-02,  1.11710691e-01]])

In [27]:
data_vector_blogger.values

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 3.1088418e-01],
       ...,
       [4.0930000e+03, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [4.0940000e+03, 0.0000000e+00, 1.4085859e-02, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [4.0950000e+03, 2.6193947e-01, 0.0000000e+00, ..., 7.1223160e-01,
        0.0000000e+00, 0.0000000e+00]])